# Getting Started with PyMapGIS

Welcome to the PyMapGIS Jupyter environment! This notebook will help you get started with spatial data analysis using PyMapGIS.

## 1. Import Libraries

Let's start by importing the essential libraries for spatial analysis:

In [ ]:
import pymapgis as pmg
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

print("✅ Libraries imported successfully!")
print(f"PyMapGIS version: {pmg.__version__}")

## 2. Create Sample Spatial Data

Let's create some sample spatial data to work with:

In [ ]:
# Create sample points around Little Rock, Arkansas
np.random.seed(42)

# Generate random points
n_points = 100
lons = np.random.normal(-92.2896, 0.05, n_points)  # Around Little Rock longitude
lats = np.random.normal(34.7465, 0.05, n_points)   # Around Little Rock latitude

# Create GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(lons, lats)]
gdf = gpd.GeoDataFrame({
    'id': range(n_points),
    'value': np.random.randint(1, 100, n_points),
    'category': np.random.choice(['A', 'B', 'C'], n_points)
}, geometry=geometry, crs='EPSG:4326')

print(f"Created GeoDataFrame with {len(gdf)} points")
print("\nFirst 5 rows:")
gdf.head()

## 3. Basic Spatial Operations

Let's perform some basic spatial operations:

In [ ]:
# Calculate the bounding box
bounds = gdf.total_bounds
print(f"Bounding box: {bounds}")

# Calculate the centroid
centroid = gdf.geometry.centroid.iloc[0]
print(f"Centroid: {centroid}")

# Project to UTM for distance calculations
gdf_utm = gdf.to_crs('EPSG:26915')  # UTM Zone 15N
print(f"\nProjected to UTM. Sample coordinates:")
print(gdf_utm.geometry.iloc[0])

## 4. Create an Interactive Map

Let's visualize our data on an interactive map:

In [ ]:
# Create an interactive map
m = gdf.explore(
    column='category',
    cmap='viridis',
    tooltip=['id', 'value', 'category'],
    popup=['id', 'value', 'category'],
    style_kwds={'radius': 5},
    tiles='CartoDB positron'
)

# Display the map
m

## 5. PyMapGIS Vector Operations

Now let's use some PyMapGIS-specific functionality:

In [ ]:
# Use PyMapGIS vector operations
from pymapgis.vector import buffer_points, calculate_distances

# Create buffers around points
try:
    buffered = buffer_points(gdf_utm, distance=1000)  # 1km buffer
    print(f"Created buffers for {len(buffered)} points")
    
    # Convert back to WGS84 for mapping
    buffered_wgs84 = buffered.to_crs('EPSG:4326')
    
    # Create map with buffers
    m_buffer = buffered_wgs84.explore(
        color='red',
        alpha=0.3,
        tiles='OpenStreetMap'
    )
    
    # Add original points
    m_buffer = gdf.explore(
        m=m_buffer,
        color='blue',
        marker_kwds={'radius': 3}
    )
    
    m_buffer
    
except Exception as e:
    print(f"Buffer operation not available: {e}")
    print("This is normal - some PyMapGIS functions may not be available in this demo")

## 6. Basic Statistics

Let's calculate some basic spatial statistics:

In [ ]:
# Basic statistics by category
stats = gdf.groupby('category')['value'].agg(['count', 'mean', 'std'])
print("Statistics by category:")
print(stats)

# Plot histogram
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
gdf['value'].hist(bins=20, alpha=0.7)
plt.title('Distribution of Values')
plt.xlabel('Value')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
gdf['category'].value_counts().plot(kind='bar')
plt.title('Count by Category')
plt.xlabel('Category')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

## 7. Next Steps

Now that you've got the basics down, try exploring:

1. **Spatial DBSCAN Example** - Check out the `spatial_dbscan_example.ipynb` notebook
2. **Real Data** - Load your own spatial datasets
3. **PyMapGIS ML** - Explore machine learning capabilities
4. **Advanced Visualization** - Create more complex maps and plots

### Useful PyMapGIS Modules:

- `pymapgis.ml` - Machine learning algorithms
- `pymapgis.vector` - Vector operations
- `pymapgis.raster` - Raster processing
- `pymapgis.viz` - Advanced visualization
- `pymapgis.io` - Data input/output

Happy mapping! 🗺️